In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Missing Data').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/06 16:32:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('ContainsNull.csv', header=True, inferSchema=True)

In [4]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [6]:
# drop the row which has null value
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [7]:
# drop the row except which has atleast 2 non null values
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [8]:
#parameter any and all
#any = drop the row which has any null value
# all = drop the row which has all null value
df.na.drop(how = 'any').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [9]:
df.na.drop(how='all').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
# subset, drop specific column has null value, All rows in sales column are removed which has null
df.na.drop(subset = 'Sales').show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [15]:
#fill the null values which some other value
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [17]:
df.na.fill('FILL Value').show() # it fills all the null values which are of string type

+----+----------+-----+
|  Id|      Name|Sales|
+----+----------+-----+
|emp1|      John| null|
|emp2|FILL Value| null|
|emp3|FILL Value|345.0|
|emp4|     Cindy|456.0|
+----+----------+-----+



In [19]:
df.na.fill(0).show() # it fills all the null of number type with 0

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [21]:
#fill data in a specific value
df.na.fill('only in Name column', subset=['Name']).show()

+----+-------------------+-----+
|  Id|               Name|Sales|
+----+-------------------+-----+
|emp1|               John| null|
|emp2|only in Name column| null|
|emp3|only in Name column|345.0|
|emp4|              Cindy|456.0|
+----+-------------------+-----+



In [38]:
#fill missing value with average value in Sales
from pyspark.sql.functions import mean
mean_val = df.select(mean(df['Sales'])).collect()
#mean_val[0].asDict()['avg(Sales)'] #converting row into Dict
mean_result = mean_val[0][0]
df.na.fill(mean_result, subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [40]:
# above in 1 line
df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], ['Sales']).show()
#df.na.fill(df.select(mean(df['Sales'])).collect()[0][0], subset=['Sales']).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

